# Лабораторная работа 2. Метод ближайших соседей и решающие деревья.

ФИО: Николаев Владимир Владимирович

Группа: 317

In [1]:
import numpy as np
import pandas as pd
import scipy
from sklearn import metrics
import sklearn.preprocessing

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.94210992096188473

In [5]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

In [7]:
y_train, y_test = y_train.values, y_test.values

## Часть 1: kNN и категориальные признаки

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре. Реализуйте самостоятельно метод k ближайших соседей, который будет уметь работать с этими функциями расстояния. Подсчитайте для каждой из них качество на тестовой выборке `X_test` при числе соседей $k = 10$. Метрика качества — AUC-ROC.

Какая функция расстояния оказалась лучшей?

In [8]:
def label_encode_features(X_train, X_test):
    for column_name in X_train.columns:
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(pd.concat((X_train, X_test))[column_name])
        X_train[column_name] = le.transform(X_train[column_name])
        X_test[column_name] = le.transform(X_test[column_name])    

In [9]:
def dst1(X, Z, X_train):
    dst = np.empty((X.shape[0], Z.shape[0]))
    for i in xrange(X.shape[0]):
        dst[i, :] = np.sum(X[i, :] != Z, axis=1)
    
    return dst


def dst2(X, Z, X_train):
    dst = np.zeros((np.size(X, axis=0), np.size(Z, axis=0)))
    l = 1.*X_train.shape[0]
    
    pX = np.empty(X_train.shape[1], dtype=np.ndarray)
    pX_counts = np.empty(X_train.shape[1], dtype=np.ndarray)
    pX_sq = np.empty(X_train.shape[1], dtype=np.ndarray)
    pX_sq_counts = np.empty(X_train.shape[1], dtype=np.ndarray)
    
    for j in xrange(X_train.shape[1]):
        pX[j] = np.array(np.unique(X_train[:, j], return_counts=True)[0], dtype=np.int32)
        pX_counts[j] = np.array(np.unique(X_train[:, j], return_counts=True)[1], dtype=np.int32)
        pX_sq[j] = np.array(np.unique(X_train[:, j], return_counts=True)[0], dtype=np.float32)
        pX_sq_counts[j] = np.array(np.unique(X_train[:, j], return_counts=True)[1], dtype=np.float32)
        pX[j][1] = pX[j][1].astype(np.float32) / l
        pX_sq_counts[j] = pX_sq_counts[j]*(pX_sq_counts[j]-1)/l/(l-1)
    
    
    for i in xrange(X.shape[0]):
        for j in xrange(X.shape[1]):
            w = pX[j] == X[i, j]
            if np.sum(w) == 1:
                dst[i, :] += (Z[:, j] == X[i, j]).T*np.sum(pX_sq_counts[j][pX_counts[j] < pX_counts[j][w]])
        dst[i, :] += np.sum(X[i, :] != Z, axis=1)
    return dst
        

def dst3(X, Z, X_train):
    dst = np.empty((np.size(X, axis=0), np.size(Z, axis=0)))
    
    f = []
    
    for i in xrange(X_train.shape[1]):
        f.append(1.0*np.bincount(X_train[:, i]))
    for i in xrange(X_train.shape[1]):
        f[i].resize(max(np.max(X[:, i]), np.max(Z[:, i])) + 1)
    
    log_fX = np.empty_like(X, dtype=np.float32)
    log_fZ = np.empty_like(Z, dtype=np.float32)
    
    for i in xrange(X.shape[1]):
        log_fX[:, i] = np.log(f[i][X[:, i]] + 1)
    for i in xrange(Z.shape[1]):
        log_fZ[:, i] = np.log(f[i][Z[:, i]] + 1)
    
    for i in xrange(X.shape[0]):
        dst[i, :] = np.sum((X[i, :] != Z)*log_fX[i, :]*log_fZ, axis=1)
    
    return dst

In [10]:
class Knn:
    def fit(self, X, y, k=1, dst_func=dst1):
        self.X = np.copy(X)
        self.y = np.copy(y)
        self.k = k
        self.dst_func = dst_func
        
        self.l = np.size(X, axis=0)
        
    def predict(self, X_test):
        l_test = X_test.shape[0]
        dst = self.dst_func(X_test, self.X, self.X)
        y_pred = np.empty(l_test)   
        for i in range(l_test):
            y_pred[i] = np.mean(self.y[dst[i, :].argsort()][:self.k])
        return y_pred

In [11]:
label_encode_features(X_train, X_test)

In [12]:
dst_funcs = [dst1, dst2, dst3]
for n, dst_func in enumerate(dst_funcs):
    model = Knn()
    model.fit(X_train.values, y_train, k=10, dst_func=dst_func)
    y_pred = model.predict(X_test.values)
    auc = metrics.roc_auc_score(y_test, y_pred)
    print "Distance function:", n+1,"k = 10 AUC:", auc
    

Distance function: 1 k = 10 AUC: 0.83088009598
Distance function: 2 k = 10 AUC: 0.836556363537
Distance function: 3 k = 10 AUC: 0.82094907632


Лучшей оказалась вторая функция расстояния (сглаженный индикатор совпадения).

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось достичь?

In [13]:
def parametr_tuning(X_train, y_train, X_test, y_test, param_grid):
    model = Knn()
    for k in param_grid["k"]:
        model.fit(X_train.values, y_train, k=k, dst_func=param_grid["dst_func"][1])
        y_pred = model.predict(X_test.values)
        auc = metrics.roc_auc_score(y_test, y_pred)
        print "Distance function:", param_grid["dst_func"][0],"k =", k ,"AUC:", auc

In [14]:
parametr_tuning(X_train, y_train, X_test, y_test, {"k": [1, 2, 4, 8, 16, 32, 64], "dst_func": ["dst1", dst1]})

Distance function: dst1 k = 1 AUC: 0.709392461188
Distance function: dst1 k = 2 AUC: 0.777446570068
Distance function: dst1 k = 4 AUC: 0.814440745988
Distance function: dst1 k = 8 AUC: 0.830774695372
Distance function: dst1 k = 16 AUC: 0.828001122835
Distance function: dst1 k = 32 AUC: 0.805931691061
Distance function: dst1 k = 64 AUC: 0.778917146675


In [15]:
parametr_tuning(X_train, y_train, X_test, y_test, {"k": np.arange(4, 20), "dst_func": ["dst1", dst1]})

Distance function: dst1 k = 4 AUC: 0.814440745988
Distance function: dst1 k = 5 AUC: 0.822305895749
Distance function: dst1 k = 6 AUC: 0.82708243927
Distance function: dst1 k = 7 AUC: 0.828995914087
Distance function: dst1 k = 8 AUC: 0.830774695372
Distance function: dst1 k = 9 AUC: 0.829924122344
Distance function: dst1 k = 10 AUC: 0.83088009598
Distance function: dst1 k = 11 AUC: 0.831201958708
Distance function: dst1 k = 12 AUC: 0.829705773258
Distance function: dst1 k = 13 AUC: 0.83102602146
Distance function: dst1 k = 14 AUC: 0.830949015219
Distance function: dst1 k = 15 AUC: 0.828103648065
Distance function: dst1 k = 16 AUC: 0.828001122835
Distance function: dst1 k = 17 AUC: 0.824370327959
Distance function: dst1 k = 18 AUC: 0.825269423005
Distance function: dst1 k = 19 AUC: 0.824044781151


In [16]:
parametr_tuning(X_train, y_train, X_test, y_test, {"k": [1, 2, 4, 8, 16, 32, 64], "dst_func": ["dst2", dst2]})

Distance function: dst2 k = 1 AUC: 0.706829420302
Distance function: dst2 k = 2 AUC: 0.771666699015
Distance function: dst2 k = 4 AUC: 0.819613462812
Distance function: dst2 k = 8 AUC: 0.839621607322
Distance function: dst2 k = 16 AUC: 0.824788426025
Distance function: dst2 k = 32 AUC: 0.808739318866
Distance function: dst2 k = 64 AUC: 0.778007089247


In [17]:
parametr_tuning(X_train, y_train, X_test, y_test, {"k": np.arange(4, 20), "dst_func": ["dst2", dst2]})

Distance function: dst2 k = 4 AUC: 0.819613462812
Distance function: dst2 k = 5 AUC: 0.829940296351
Distance function: dst2 k = 6 AUC: 0.836720799269
Distance function: dst2 k = 7 AUC: 0.837461119486
Distance function: dst2 k = 8 AUC: 0.839621607322
Distance function: dst2 k = 9 AUC: 0.838259755981
Distance function: dst2 k = 10 AUC: 0.836556363537
Distance function: dst2 k = 11 AUC: 0.835299373671
Distance function: dst2 k = 12 AUC: 0.832711263075
Distance function: dst2 k = 13 AUC: 0.831060166585
Distance function: dst2 k = 14 AUC: 0.827797150643
Distance function: dst2 k = 15 AUC: 0.825991502536
Distance function: dst2 k = 16 AUC: 0.824788426025
Distance function: dst2 k = 17 AUC: 0.824283078179
Distance function: dst2 k = 18 AUC: 0.82537733957
Distance function: dst2 k = 19 AUC: 0.821820046567


In [18]:
parametr_tuning(X_train, y_train, X_test, y_test, {"k": [1, 2, 4, 8, 16, 32, 64], "dst_func": ["dst3", dst3]})

Distance function: dst3 k = 1 AUC: 0.723262210836
Distance function: dst3 k = 2 AUC: 0.769017846039
Distance function: dst3 k = 4 AUC: 0.805659788041
Distance function: dst3 k = 8 AUC: 0.819612025122
Distance function: dst3 k = 16 AUC: 0.810324012074
Distance function: dst3 k = 32 AUC: 0.787446508967
Distance function: dst3 k = 64 AUC: 0.754381268775


In [19]:
parametr_tuning(X_train, y_train, X_test, y_test, {"k": np.arange(4, 20), "dst_func": ["dst3", dst3]})

Distance function: dst3 k = 4 AUC: 0.805659788041
Distance function: dst3 k = 5 AUC: 0.810224811501
Distance function: dst3 k = 6 AUC: 0.815499244854
Distance function: dst3 k = 7 AUC: 0.81975004331
Distance function: dst3 k = 8 AUC: 0.819612025122
Distance function: dst3 k = 9 AUC: 0.821106952594
Distance function: dst3 k = 10 AUC: 0.82094907632
Distance function: dst3 k = 11 AUC: 0.81878580296
Distance function: dst3 k = 12 AUC: 0.820109285964
Distance function: dst3 k = 13 AUC: 0.818310466882
Distance function: dst3 k = 14 AUC: 0.816922197997
Distance function: dst3 k = 15 AUC: 0.812440919949
Distance function: dst3 k = 16 AUC: 0.810324012074
Distance function: dst3 k = 17 AUC: 0.811584236741
Distance function: dst3 k = 18 AUC: 0.809703649072
Distance function: dst3 k = 19 AUC: 0.808105118103


1. Функция расстояния 1: максимальный AUC-ROC равен 0.831 при k=11
2. Функция расстояния 2: максимальный AUC-ROC равен 0.840 при k=8
3. Функция расстояния 3: максимальный AUC-ROC равен 0.821 при k=9

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `clicks` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`clicks` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `clicks` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [20]:
from sklearn.cross_validation import KFold

In [21]:
def convert_feature(feature, X_train, target, X_test, test_index):
    feature_column = X_train[feature]
    
    if test_index is None:
        test_index = np.arange(X_test.shape[0])
    
    bincount = np.bincount(feature_column.astype(np.int64))
    bincount.resize(np.max(X_test[feature])+1)
    
    bincount_ones = np.bincount(feature_column[target == 1].astype(np.int64))
    bincount_ones.resize(np.max(X_test[feature])+1)
     
    X_test[feature+"_counts"].iloc[test_index] = 1.0*bincount[X_test[feature].iloc[test_index].values.astype(np.int64)]/feature_column.shape[0]
    X_test[feature+"_clicks"].iloc[test_index] = 1.0*bincount_ones[X_test[feature].iloc[test_index].values.astype(np.int64)]/feature_column.shape[0]   
        
def categories_to_counters(X_train_, X_test_, y_train_, folding=True, inplace=False):  
    X_train = X_train_.copy(deep=True)
    X_test = X_test_.copy(deep=True)
    
    N_FOLDS = 5
      
    old_features = np.copy(X_train.columns)
    
    for feature in old_features:
        np.random.seed(1234)
        kf = KFold(X_train.values.shape[0], n_folds=N_FOLDS, random_state=1234)
        
        X_train[feature+"_counts"] = np.empty(X_train.shape[0])
        X_train[feature+"_clicks"] = np.empty(X_train.shape[0])
        X_test[feature+"_counts"] = np.empty(X_test.shape[0])
        X_test[feature+"_clicks"] = np.empty(X_test.shape[0])
        
        if folding:
            for train_index, test_index in kf:
                convert_feature(feature, X_train.iloc[train_index], y_train[train_index], X_train, test_index)
            convert_feature(feature, X_train, y_train, X_test, test_index=None)
        else:
            convert_feature(feature, X_train, y_train, X_train, test_index=None)
            convert_feature(feature, X_train, y_train, X_test, test_index=None)
        
        X_train[feature+"_clicks_by_counts"] = (X_train[feature+"_clicks"]*X_train.shape[0]*1.*(N_FOLDS-1)/N_FOLDS+1)/(X_train[feature+"_counts"]*X_train.shape[0]*1.*(N_FOLDS-1)/N_FOLDS+2)
        X_test[feature+"_clicks_by_counts"] = (X_test[feature+"_clicks"]*X_train.shape[0]+1)/(X_test[feature+"_counts"]*X_train.shape[0]+2)
    
    X_train.drop(old_features, axis=1, inplace=True)
    X_test.drop(old_features, axis=1, inplace=True)
    
    if inplace:
        X_train_ = X_train
        X_test_ = X_test
    
    return X_train, X_test         

In [22]:
def dst_l2(X, Z, X_train):
    return scipy.spatial.distance.cdist(X, Z)

In [24]:
X_train_counters_f, X_test_counters_f = categories_to_counters(X_train, X_test, y_train, folding=True)

/home/deathcoil/anaconda3/envs/python2env/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
param_grid = {"k": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024], "dst_func": ["Euclidean distance", dst_l2]}
parametr_tuning(X_train_counters_f, y_train, X_test_counters_f, y_test, param_grid=param_grid)

Distance function: Euclidean distance k = 1 AUC: 0.657273612432
Distance function: Euclidean distance k = 2 AUC: 0.691519914515
Distance function: Euclidean distance k = 4 AUC: 0.729622189587
Distance function: Euclidean distance k = 8 AUC: 0.755632418028
Distance function: Euclidean distance k = 16 AUC: 0.782909879874
Distance function: Euclidean distance k = 32 AUC: 0.794314441375
Distance function: Euclidean distance k = 64 AUC: 0.798659138925
Distance function: Euclidean distance k = 128 AUC: 0.803383476347
Distance function: Euclidean distance k = 256 AUC: 0.79848742489
Distance function: Euclidean distance k = 512 AUC: 0.792518857094
Distance function: Euclidean distance k = 1024 AUC: 0.789330870571


In [26]:
param_grid = {"k": [64, 75, 90, 110, 128, 150, 180, 220, 256], "dst_func": ["Euclidean distance", dst_l2]}
parametr_tuning(X_train_counters_f, y_train, X_test_counters_f, y_test, param_grid=param_grid)

Distance function: Euclidean distance k = 64 AUC: 0.798659138925
Distance function: Euclidean distance k = 75 AUC: 0.800348873319
Distance function: Euclidean distance k = 90 AUC: 0.804030796027
Distance function: Euclidean distance k = 110 AUC: 0.803424989631
Distance function: Euclidean distance k = 128 AUC: 0.803383476347
Distance function: Euclidean distance k = 150 AUC: 0.801331803632
Distance function: Euclidean distance k = 180 AUC: 0.801818102092
Distance function: Euclidean distance k = 220 AUC: 0.799907233088
Distance function: Euclidean distance k = 256 AUC: 0.79848742489


In [27]:
X_train_counters_no_f, X_test_counters_no_f = categories_to_counters(X_train, X_test, y_train, folding=False)

In [28]:
param_grid = {"k": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024], "dst_func": ["Euclidean distance", dst_l2]}
parametr_tuning(X_train_counters_no_f, y_train, X_test_counters_no_f, y_test, param_grid=param_grid)

Distance function: Euclidean distance k = 1 AUC: 0.689648581971
Distance function: Euclidean distance k = 2 AUC: 0.723752283231
Distance function: Euclidean distance k = 4 AUC: 0.750699256211
Distance function: Euclidean distance k = 8 AUC: 0.759110098978
Distance function: Euclidean distance k = 16 AUC: 0.768595704471
Distance function: Euclidean distance k = 32 AUC: 0.778517109583
Distance function: Euclidean distance k = 64 AUC: 0.781826940288
Distance function: Euclidean distance k = 128 AUC: 0.791570700895
Distance function: Euclidean distance k = 256 AUC: 0.795329899412
Distance function: Euclidean distance k = 512 AUC: 0.792851861916
Distance function: Euclidean distance k = 1024 AUC: 0.788894172398


In [29]:
param_grid = {"k": [128, 150, 180, 220, 256, 300, 380, 450, 512], "dst_func": ["Euclidean distance", dst_l2]}
parametr_tuning(X_train_counters_no_f, y_train, X_test_counters_no_f, y_test, param_grid=param_grid)

Distance function: Euclidean distance k = 128 AUC: 0.791570700895
Distance function: Euclidean distance k = 150 AUC: 0.791874502649
Distance function: Euclidean distance k = 180 AUC: 0.793209756736
Distance function: Euclidean distance k = 220 AUC: 0.79506967762
Distance function: Euclidean distance k = 256 AUC: 0.795329899412
Distance function: Euclidean distance k = 300 AUC: 0.794726608972
Distance function: Euclidean distance k = 380 AUC: 0.794556152916
Distance function: Euclidean distance k = 450 AUC: 0.795015494698
Distance function: Euclidean distance k = 512 AUC: 0.792851861916


1. Счетчики с фолдингом:  максимальный AUC-ROC равен 0.804 при k=90
2. Счетчики без фолдинга: максимальный AUC-ROC равен 0.795 при k=256

Качество с фолдингом оказалось выше. Это логично, так как без фолдинга в обучающую выборку вносится информация о целевой переменной.

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $f_i$, $f_j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$. Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

In [30]:
def add_pair_features(X_train_, X_test_, inplace=False):
    X_train = X_train_.copy(deep=True)
    X_test = X_test_.copy(deep=True)
    old_features = np.copy(X_train.columns)
    
    for feature1 in old_features:
        for feature2 in old_features:
            if feature1 != feature2:
                X_train[feature1+"+"+feature2] = np.core.defchararray.add(np.core.defchararray.add(X_train[feature1].values.astype(np.str),
                                                                                                   "&"),
                                                                          X_train[feature2].values.astype(np.str))
                X_test[feature1+"+"+feature2] = np.core.defchararray.add(np.core.defchararray.add(X_test[feature1].values.astype(np.str),
                                                                                                   "&"),
                                                                         X_test[feature2].values.astype(np.str))
    if inplace:
        X_train_ = X_train.copy(deep=True)
        X_test_ = X_test.copy(deep=True)
    
    return X_train, X_test

In [31]:
X_train_pair, X_test_pair = add_pair_features(X_train, X_test)

In [32]:
label_encode_features(X_train_pair, X_test_pair)

In [33]:
X_train_pair_counters_f, X_test_pair_counters_f = categories_to_counters(X_train_pair, X_test_pair, y_train, folding=True)

In [34]:
param_grid = {"k": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024], "dst_func": ["Euclidean distance", dst_l2]}
parametr_tuning(X_train_pair_counters_f, y_train, X_test_pair_counters_f, y_test, param_grid=param_grid)

Distance function: Euclidean distance k = 1 AUC: 0.679320131031
Distance function: Euclidean distance k = 2 AUC: 0.717363011557
Distance function: Euclidean distance k = 4 AUC: 0.758150620974
Distance function: Euclidean distance k = 8 AUC: 0.792169768122
Distance function: Euclidean distance k = 16 AUC: 0.800264498918
Distance function: Euclidean distance k = 32 AUC: 0.826130868559
Distance function: Euclidean distance k = 64 AUC: 0.831865092972
Distance function: Euclidean distance k = 128 AUC: 0.835478276153
Distance function: Euclidean distance k = 256 AUC: 0.837517998075
Distance function: Euclidean distance k = 512 AUC: 0.83716468589
Distance function: Euclidean distance k = 1024 AUC: 0.836163514892


In [35]:
X_train_pair_counters_no_f, X_test_pair_counters_no_f = categories_to_counters(X_train_pair, X_test_pair, y_train, folding=False)

In [36]:
param_grid = {"k": [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024], "dst_func": ["Euclidean distance", dst_l2]}
parametr_tuning(X_train_pair_counters_no_f, y_train, X_test_pair_counters_no_f, y_test, param_grid=param_grid)

Distance function: Euclidean distance k = 1 AUC: 0.708222541389
Distance function: Euclidean distance k = 2 AUC: 0.748413060402
Distance function: Euclidean distance k = 4 AUC: 0.77401920825
Distance function: Euclidean distance k = 8 AUC: 0.785171994384
Distance function: Euclidean distance k = 16 AUC: 0.793293412291
Distance function: Euclidean distance k = 32 AUC: 0.803836078961
Distance function: Euclidean distance k = 64 AUC: 0.807065938189
Distance function: Euclidean distance k = 128 AUC: 0.81839816594
Distance function: Euclidean distance k = 256 AUC: 0.824934171794
Distance function: Euclidean distance k = 512 AUC: 0.828263680873
Distance function: Euclidean distance k = 1024 AUC: 0.827331608825


1. Счетчики с фолдингом:  максимальный AUC-ROC равен 0.838 при k=256
2. Счетчики без фолдинга: максимальный AUC-ROC равен 0.828 при k=512

## Часть 2: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

In [37]:
def parametr_tuning_tree(X_train, y_train, X_test, y_test, param_grid):
    for max_depth in param_grid["max_depth"]:
        for min_samples_leaf in param_grid["min_samples_leaf"]:
            model = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=1234)
            model.fit(X_train.values, y_train)
            y_pred = model.predict_proba(X_test.values)[:, 1]
            auc = metrics.roc_auc_score(y_test, y_pred)
            print "Max depth:", max_depth, "min_samples_leaf:", min_samples_leaf, "AUC:", auc

In [38]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
param_grid = {"max_depth" : np.arange(2, 12, 2), "min_samples_leaf": [1, 2, 4, 8, 16, 32, 64]}
parametr_tuning_tree(X_train_pair_counters_no_f, y_train, X_test_pair_counters_no_f, y_test, param_grid)

Max depth: 2 min_samples_leaf: 1 AUC: 0.50708394525
Max depth: 2 min_samples_leaf: 2 AUC: 0.50708394525
Max depth: 2 min_samples_leaf: 4 AUC: 0.50708394525
Max depth: 2 min_samples_leaf: 8 AUC: 0.50708394525
Max depth: 2 min_samples_leaf: 16 AUC: 0.58629317651
Max depth: 2 min_samples_leaf: 32 AUC: 0.58629317651
Max depth: 2 min_samples_leaf: 64 AUC: 0.585906078623
Max depth: 4 min_samples_leaf: 1 AUC: 0.543733974255
Max depth: 4 min_samples_leaf: 2 AUC: 0.543733974255
Max depth: 4 min_samples_leaf: 4 AUC: 0.560637158809
Max depth: 4 min_samples_leaf: 8 AUC: 0.552624556024
Max depth: 4 min_samples_leaf: 16 AUC: 0.629639334033
Max depth: 4 min_samples_leaf: 32 AUC: 0.606164920233
Max depth: 4 min_samples_leaf: 64 AUC: 0.588939513529
Max depth: 6 min_samples_leaf: 1 AUC: 0.557973479661
Max depth: 6 min_samples_leaf: 2 AUC: 0.552443586863
Max depth: 6 min_samples_leaf: 4 AUC: 0.582379246665
Max depth: 6 min_samples_leaf: 8 AUC: 0.583365142213
Max depth: 6 min_samples_leaf: 16 AUC: 0.62966

Максимальный AUC-ROC равен 0.630 при max_depth = 6, min_samples_leaf = 16

#### 2. Настройте случайный лес, подобрав оптимальное число деревьев `n_estimators`. Какое качество на тестовой выборке он дает?

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
def parametr_tuning_forest(X_train, y_train, X_test, y_test, param_grid):
    for n_estimators in param_grid["n_estimators"]:
        model = RandomForestClassifier(n_estimators=n_estimators, random_state=1234)
        model.fit(X_train.values, y_train)
        y_pred = model.predict_proba(X_test.values)[:, 1]
        auc = metrics.roc_auc_score(y_test, y_pred)
        print "# trees:", n_estimators, "AUC:", auc

In [42]:
param_grid = {"n_estimators" : [2, 10, 25, 100, 200, 500, 1000]}
parametr_tuning_forest(X_train_pair_counters_no_f, y_train, X_test_pair_counters_no_f, y_test, param_grid)

# trees: 2 AUC: 0.604458921977
# trees: 10 AUC: 0.668080274829
# trees: 25 AUC: 0.703006963449
# trees: 100 AUC: 0.718288883569
# trees: 200 AUC: 0.715229210831
# trees: 500 AUC: 0.723840970699
# trees: 1000 AUC: 0.72835297936


Максимальный AUC-ROC равен 0.728 при n_estimators = 1000. Понятно, что с увеличением количества деревьев можно еще улучшить качество.

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

In [43]:
param_grid = {"n_estimators" : [2, 10, 25, 100, 200, 500, 1000]}
parametr_tuning_forest(X_train_pair_counters_f, y_train, X_test_pair_counters_f, y_test, param_grid)

# trees: 2 AUC: 0.725071632877
# trees: 10 AUC: 0.812219695483
# trees: 25 AUC: 0.838099543462
# trees: 100 AUC: 0.849971264182
# trees: 200 AUC: 0.855345347385
# trees: 500 AUC: 0.858254152586
# trees: 1000 AUC: 0.862260274268


Максимальный AUC-ROC равен 0.862 при n_estimators = 1000. При увеличении количества деревьев качество также можно улучшить.

Результат деревьев на данных с фолдингом намного лучше, чем без фолдинга. Логично, что леса на счетчиках без фолдинга сильно переобучаются, так как по признакам "clicks" будет хорошее разделение на обучении из-за внесенной информации из целевой переменной.